In [2]:
import numpy as np
import magpylib as magpy
from scipy.optimize import minimize, Bounds
from magpylib.magnet import Cuboid, Cylinder, CylinderSegment
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid
from maggeometry import n_rings
from alive_progress import alive_bar

### Nelder Mead Adaptive Optimization of 2 rings
All variables can be optimized independently for each ring set

1. Run 50 shots using Nelder-Mead with nonuniformity cost function
2. Optimize for field strength using new cost function

In [2]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror_z = True
base_ring_config = [Br, mag_dir, mirror_z]
# innerrad, width, thickness, dist
bounds = ((1, 150), (2, 20), (2, 20), (1, 300))

### Definition of 2 ring objective function

In [8]:
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
def obj1_nonuniformity(x):
    obj_ring_config=[[*base_ring_config, *x]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity

In [5]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from alive_progress import alive_bar

options = {'disp': True, 'fatol': 1e-12, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 50
results_x = []
results_fun = []
guesses_x0 = []
with alive_bar(shots, force_tty=True) as bar:
    for i in range(shots):
        x0 = np.zeros(len(bounds))
        for b in range(len(bounds)):
            x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
        guesses_x0.append(x0)
        results_x.append(res.x)
        results_fun.append(res.fun)
        bar()

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

on 0: Optimization terminated successfully.                                     
on 0:          Current function value: 0.000010                                 
on 0:          Iterations: 468                                                  
on 0:          Function evaluations: 824                                        
on 1: Optimization terminated successfully.                                     
on 1:          Current function value: 0.000010                                 
on 1:          Iterations: 603                                                  
on 1:          Function evaluations: 1039                                       
on 2: Optimization terminated successfully.                                     
on 2:          Current function value: 0.000010                                 
on 2:          Iterations: 408                                                  
on 2:          Function evaluations: 709                                        
on 3: Optimization terminate

on 28: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_54136/2633242801.py:19: RuntimeWarning: Maximum number of iterations has been exceeded.
         res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


on 29: Optimization terminated successfully.                                    
on 29:          Current function value: 0.000001                                
on 29:          Iterations: 583                                                 
on 29:          Function evaluations: 1040                                      
on 30: Optimization terminated successfully.                                    
on 30:          Current function value: 0.000010                                
on 30:          Iterations: 221                                                 
on 30:          Function evaluations: 399                                       
on 31: Optimization terminated successfully.                                    
on 31:          Current function value: 0.000001                                
on 31:          Iterations: 239                                                 
on 31:          Function evaluations: 448                                       
on 32: Optimization terminat

In [6]:
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results_g_center = []
for i in results_x:
    magnets = n_rings([[*base_ring_config, *i]])
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results_g_center.append(center_field)
print(results_g_center)

[9.342139795324265, 21.700471411974718, 9.428798765265274, 27.373629121720633, 10.328184444492194, 2.8196254097439333, 23.683850423649897, 0.24217355208612457, 23.683850423649897, 3.489994322678034, 23.683850423649897, 23.683850423649897, 23.683850423649897, 0.35266587228808344, 3.6514757215721083, 10.684360076528439, 2.367731438545133, 2.743800621481327, 20.489052974190223, 23.683850423649897, 92.60878493528637, 1.878289832108333, 9.337189196494045, 23.683850423649897, 92.56288744750336, 2.8969036903317402, 4.588495988306391, 2.747991062305739, 0.3231928811444629, 22.568672501659766, 31.478774346531466, 1.634448465395868, 5.02934239988555, 1.063897725396763, 23.683850423649897, 2.6627830397325702, 9.337182950587195, 2.743802448146835, 2.42275868345061, 23.683850423649897, 3.5063945709891797, 2.42275868345061, 2.42275868345061, 19.002293995065056, 92.6213257797832, 2.367731438545133, 0.7374186934059557, 3.698751627030532, 4.749799579810774, 23.683850423649897]


In [7]:
from maghelper import make_opt_res_csv
make_opt_res_csv('results/2 rings/2023_07_13_adaptive_stage1opt_2rings_1090T_', results_fun, results_g_center, results_x, guesses_x0)

Number of unique results: 38
    nonuniformity  center_field_gauss  r_1_innerrad  r_1_width  r_1_thickness  \
0    7.959436e-07           27.373629    150.000000  20.000000      20.000000   
1    7.963403e-07           22.568673    149.999997  19.999877      16.476879   
2    7.970285e-07           10.684360    149.999836  19.996556       7.791782   
3    7.971424e-07            4.749800    150.000000  19.999979       3.462370   
4    7.971587e-07            3.651476    149.999973  19.999915       2.661675   
5    7.971587e-07            3.489994    150.000000  19.999988       2.543949   
6    7.971647e-07            2.896904    150.000000  20.000000       2.111603   
7    7.971664e-07            2.743801    150.000000  19.999981       2.000000   
8    7.971666e-07            2.743802    150.000000  20.000000       2.000000   
9    7.974538e-07            5.029342    149.999997  19.968489       3.671238   
10   7.976520e-07            2.747991    149.999995  19.951876       2.007285   

(array([[7.95943632e-07, 2.73736291e+01, 1.50000000e+02, 2.00000000e+01,
         2.00000000e+01, 2.71023521e+02, 2.00884726e+01, 8.07068847e+00,
         1.03286432e+01, 1.23538903e+02],
        [7.96340306e-07, 2.25686725e+01, 1.49999997e+02, 1.99998774e+01,
         1.64768788e+01, 2.70862531e+02, 9.79063678e+01, 1.86124884e+01,
         1.91713281e+01, 1.95619757e+02],
        [7.97028547e-07, 1.06843601e+01, 1.49999836e+02, 1.99965562e+01,
         7.79178180e+00, 2.70595799e+02, 1.32035666e+02, 1.76163965e+01,
         8.58178999e+00, 2.54615144e+02],
        [7.97142426e-07, 4.74979958e+00, 1.50000000e+02, 1.99999794e+01,
         3.46237038e+00, 2.70537780e+02, 9.90104916e+01, 1.73773837e+01,
         5.98334815e+00, 2.62115583e+02],
        [7.97158720e-07, 3.65147572e+00, 1.49999973e+02, 1.99999150e+01,
         2.66167542e+00, 2.70531545e+02, 1.20233619e+02, 8.12736127e+00,
         8.32943140e+00, 1.90826213e+02],
        [7.97158726e-07, 3.48999432e+00, 1.50000000e+02, 1.9

### Define 2nd stage multiobjective cost function
Only use results that are within +20% of minimum homogeneity for stage 2

In [2]:
import numpy as np
import pandas as pd

# guess_threshold = 1.2 # only use results that are within +20% of minimum homogeneity for stage 2

st1 = pd.read_csv('results/2 rings/2023_07_13_adaptive_stage1opt_2rings_1090T_38.csv')
st1 = np.delete(st1.to_numpy(), 0, axis=1) #remove first column of indices
min_nonun = st1[0][0]
# nonun_threshold = min_nonun*guess_threshold
nonun_threshold = 1e-6
print("Nonuniformity threshold:", nonun_threshold)
st2g = st1[st1[:,0] < nonun_threshold]

Nonuniformity threshold: 1e-06


array([[7.95943632e-07, 2.73736291e+01, 1.50000000e+02, 2.00000000e+01,
        2.00000000e+01, 2.71023521e+02, 2.00884726e+01, 8.07068847e+00,
        1.03286432e+01, 1.23538903e+02],
       [7.96340306e-07, 2.25686725e+01, 1.49999997e+02, 1.99998774e+01,
        1.64768788e+01, 2.70862531e+02, 9.79063678e+01, 1.86124884e+01,
        1.91713281e+01, 1.95619757e+02],
       [7.97028547e-07, 1.06843601e+01, 1.49999836e+02, 1.99965562e+01,
        7.79178180e+00, 2.70595799e+02, 1.32035666e+02, 1.76163965e+01,
        8.58178999e+00, 2.54615144e+02],
       [7.97142426e-07, 4.74979958e+00, 1.50000000e+02, 1.99999794e+01,
        3.46237038e+00, 2.70537780e+02, 9.90104916e+01, 1.73773837e+01,
        5.98334815e+00, 2.62115583e+02],
       [7.97158720e-07, 3.65147572e+00, 1.49999973e+02, 1.99999150e+01,
        2.66167542e+00, 2.70531545e+02, 1.20233619e+02, 8.12736127e+00,
        8.32943140e+00, 1.90826213e+02],
       [7.97158726e-07, 3.48999432e+00, 1.50000000e+02, 1.99999882e+01,
   

In [3]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror_z = True
base_ring_config = [Br, mag_dir, mirror_z]
# innerrad, width, thickness, dist
bounds = ((1, 150), (2, 20), (2, 20), (1, 300))

In [4]:
# 10G target
field_strength_target = 10

grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
alpha = 0.2
def obj2_nonuniformity(x):
    obj_ring_config=[[*base_ring_config, *x]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    cost = av_nonuniformity*abs(center_field - field_strength_target)
#     cost = alpha*av_nonuniformity * 1e6 + (1-alpha)*abs(center_field - field_strength_target)
    return cost

### Optimize 2 rings 2nd stage

In [5]:
results2_x = []
results2_fun = []
method2 = 'Nelder-Mead'
options2 = {'disp': True, 'fatol': 1e-17, 'maxiter': 5e3, 'adaptive': True}
with alive_bar(len(st2g), force_tty=True) as bar:
    for s0 in st2g:
        res2 = minimize(obj2_nonuniformity, s0[2:6], method=method2, options=options2, bounds=bounds)
        results2_x.append(res2.x)
        results2_fun.append(res2.fun)
        bar()
        
print(results2_x)
print(results2_fun)

on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_54289/3678240242.py:7: RuntimeWarning: Maximum number of iterations has been exceeded.
        res2 = minimize(obj2_nonuniformity, s0[2:6], method=method2, options=options2, bounds=bounds)


on 1: Optimization terminated successfully.                                     
on 1:          Current function value: 0.000000                                 
on 1:          Iterations: 247                                                  
on 1:          Function evaluations: 444                                        
on 2: Optimization terminated successfully.                                     
on 2:          Current function value: 0.000000                                 
on 2:          Iterations: 246                                                  
on 2:          Function evaluations: 487                                        
on 3: Optimization terminated successfully.                                     
on 3:          Current function value: 0.000000                                 
on 3:          Iterations: 242                                                  
on 3:          Function evaluations: 439                                        
on 4: Optimization terminate

In [8]:
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2_nonun = []
results2_g_center = []
for x in results2_x:
    obj_ring_config=[[*base_ring_config, *x]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results2_nonun.append(obj1_nonuniformity(x))
    results2_g_center.append(center_field)

print(results2_nonun)
print(results2_g_center)

[7.959436209682873e-07, 9.637272464051714e-07, 0.00013966134394204617, 8.84946921025474e-07, 9.607351869859706e-07, 9.122361755908598e-07, 8.440085660408514e-07, 8.436268176425673e-07, 8.429630846911517e-07, 7.980525500190361e-07, 9.782581122666639e-07, 9.75241120034939e-07, 7.973901291166383e-07, 8.418790197496415e-07, 8.620416844220707e-07, 1.0639857747813601e-06, 1.3763165366226638e-06, 2.9387785499069048e-06, 3.336012076657815e-06, 9.673193132487536e-07, 1.974021017103268e-06, 1.1360743517888581e-06]
[27.37362655244244, 9.99999999998499, 10.000000000000382, 10.000000000012719, 10.00000000002042, 9.999999999980371, 9.999999999981899, 10.000000000017346, 9.999999999983453, 10.000000000026592, 9.999999999980362, 9.999999999984981, 10.0000000000158, 9.999999999981908, 10.000000000011191, 9.999999999986535, 9.99999999998959, 10.00000000000039, 9.9999999999973, 10.000000000012719, 10.000000000012719, 9.9999999999865]


In [9]:
def make_opt_res2_csv(name, nonun, g_center, x, fun):
    r_f = np.array([fun])
    r_n = np.array([nonun])
    r_gc = np.array([g_center])
    r_x = np.array(x)

    a = np.concatenate((r_n.T, r_gc.T, r_x, r_f.T), axis=1)
    b = a[a[:, 0].argsort()]

    columns = ['nonuniformity', 'center_field_gauss']
    n = int(b.shape[1]) - len(columns)
    # need 4 specs to define a ring: innerrad, width, thickness, dist
    n_ring_sets = int(n/ 4)
    
    for i in range(1, n_ring_sets+1):
        columns.append('r_' + str(i) + '_innerrad')
        columns.append('r_' + str(i) + '_width')
        columns.append('r_' + str(i) + '_thickness')
        columns.append('r_' + str(i) + '_dist')
    
    columns.append('objective')
    
    columns = np.array(columns)
    b_df = pd.DataFrame(b, columns = columns)
    
    c = np.concatenate((r_n.T, r_x), axis=1)
    new_array = [tuple(row) for row in c]
    u = np.unique(new_array, axis=0)
    n_unique = len(u)
    print('Number of unique results:', n_unique)
    print(b_df)
    b_df.to_csv(name + str(n_unique) + '.csv')
    return b, b_df


In [10]:
from maghelper import make_opt_res2_csv 
make_opt_res2_csv('results/2 rings/2023_07_13_adaptive_stage2opt_2rings_1090T_', results2_nonun, results2_g_center, results2_x, results2_fun)

Number of unique results: 22
    nonuniformity  center_field_gauss  r_1_innerrad  r_1_width  r_1_thickness  \
0    7.959436e-07           27.373627    150.000000  20.000000      20.000000   
1    7.973901e-07           10.000000    150.000000  19.963117       7.303095   
2    7.980526e-07           10.000000    149.999973  19.988032       7.294967   
3    8.418790e-07           10.000000    149.999941  19.571384       7.430387   
4    8.429631e-07           10.000000    150.000000  20.000000       7.290414   
5    8.436268e-07           10.000000    150.000000  19.999602       7.290535   
6    8.440086e-07           10.000000    150.000000  20.000000       7.290404   
7    8.620417e-07           10.000000    149.999997  17.640173       8.145608   
8    8.849469e-07           10.000000    150.000000  19.999810       7.290128   
9    9.122362e-07           10.000000    150.000000  19.999741       7.289962   
10   9.607352e-07           10.000000    149.999961  19.998779       7.289973   

(array([[7.95943621e-07, 2.73736266e+01, 1.50000000e+02, 2.00000000e+01,
         2.00000000e+01, 2.71023541e+02, 1.38284272e-05],
        [7.97390129e-07, 1.00000000e+01, 1.50000000e+02, 1.99631174e+01,
         7.30309524e+00, 2.70559669e+02, 1.25993175e-17],
        [7.98052550e-07, 1.00000000e+01, 1.49999973e+02, 1.99880322e+01,
         7.29496722e+00, 2.70575560e+02, 2.12218628e-17],
        [8.41879020e-07, 1.00000000e+01, 1.49999941e+02, 1.95713838e+01,
         7.43038736e+00, 2.70218460e+02, 1.52314389e-17],
        [8.42963085e-07, 1.00000000e+01, 1.50000000e+02, 2.00000000e+01,
         7.29041437e+00, 2.70564171e+02, 1.39483112e-17],
        [8.43626818e-07, 1.00000000e+01, 1.50000000e+02, 1.99996021e+01,
         7.29053518e+00, 2.70563667e+02, 1.46336558e-17],
        [8.44008566e-07, 1.00000000e+01, 1.50000000e+02, 2.00000000e+01,
         7.29040443e+00, 2.70563883e+02, 1.52774634e-17],
        [8.62041684e-07, 1.00000000e+01, 1.49999997e+02, 1.76401727e+01,
         8

In [4]:
import numpy as np
import pandas as pd

# guess_threshold = 1.2 # only use results that are within +20% of minimum homogeneity for stage 2

st1 = pd.read_csv('results/2 rings/2023_07_13_adaptive_stage1opt_2rings_1090T_38.csv')
st1 = np.delete(st1.to_numpy(), 0, axis=1) #remove first column of indices
min_nonun = st1[0][0]
# nonun_threshold = min_nonun*guess_threshold
nonun_threshold = 1e-6
print("Nonuniformity threshold:", nonun_threshold)
st2g = st1[st1[:,0] < nonun_threshold]

# 10G target
field_strength_target = 10
Br = 1.09e3
mag_dir = (0,0,1)
mirror_z = True
base_ring_config = [Br, mag_dir, mirror_z]
# innerrad, width, thickness, dist
bounds2 = ((1, 150), (2, 20), (2, 20), (1, 300))

grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
alpha = 0.2
def obj2_nonuniformity(x):
    obj_ring_config=[[*base_ring_config, *x]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
#     cost = av_nonuniformity*abs(center_field - field_strength_target)
    cost = alpha*av_nonuniformity * 1e6 + (1-alpha)*abs(center_field - field_strength_target)
    return cost

results2_x = []
results2_fun = []
method2 = 'Nelder-Mead'
options2 = {'disp': True, 'fatol': 1e-12, 'maxiter': 5e3, 'adaptive': True}
with alive_bar(len(st2g), force_tty=True) as bar:
    for s0 in st2g:
        res2 = minimize(obj2_nonuniformity, s0[2:6], method=method2, options=options2, bounds=bounds2)
        results2_x.append(res2.x)
        results2_fun.append(res2.fun)
        bar()
        
print(results2_x)
print(results2_fun)

Nonuniformity threshold: 1e-06


on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_54427/2064920539.py:40: RuntimeWarning: Maximum number of iterations has been exceeded.
        res2 = minimize(obj2_nonuniformity, s0[2:6], method=method2, options=options2, bounds=bounds2)


|█████▌⚠︎                                 | (!) 3/22 [14%] in 31:03.0 (0.00/s)   


KeyboardInterrupt: 

In [9]:
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2_nonun = []
results2_g_center = []
print(results2_x)
print(results2_fun)
for x in results2_x:
    obj_ring_config=[[*base_ring_config, *x]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results2_nonun.append(obj1_nonuniformity(x))
    results2_g_center.append(center_field)

print(results2_nonun)
print(results2_g_center)

[array([150.        ,  20.        ,  20.        , 271.02624605]), array([149.999999  ,  19.99985767,   7.29131183, 270.58872114]), array([149.99998338,  19.99667535,   7.29234498, 270.58663141])]
[14.057950947068836, 0.15940514441094852, 0.15940954011131594]
[7.966516559649687e-07, 7.970256867996016e-07, 7.970476826015555e-07]
[27.373275769844803, 10.000000008813785, 10.000000004488756]
